In [25]:
from abc import ABC, abstractmethod
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Union, Optional
from tqdm import tqdm
from sklearn.metrics import ndcg_score
from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
import numpy as np

In [26]:
# Абстрактный класс для моделей эмбеддинга
class EmbeddingModel(ABC):
    @abstractmethod
    def compute_embeddings(self, texts: List[str]) -> Union[List[List[float]], np.ndarray]:
        """Вычисляет эмбеддинги для списка текстов."""
        pass
    
    @abstractmethod
    def compute_single_embedding(self, text: str) -> Union[List[float], np.ndarray]:
        """Вычисляет эмбеддинг для одного текста."""
        pass
    
    @abstractmethod
    def get_vector_size(self) -> int:
        """Возвращает размерность векторов для модели."""
        pass
    
    @abstractmethod
    def get_model_name(self) -> str:
        """Возвращает имя модели для отчетов."""
        pass

# Реализация для SentenceTransformer
class SentenceTransformerModel(EmbeddingModel):
    def __init__(self, model_name: str = 'BAAI/bge-large-en-v1.5'):
        self.model_name = model_name
        self.model = SentenceTransformer(model_name)
    
    def compute_embeddings(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts, normalize_embeddings=True)
        return [embedding.tolist() for embedding in embeddings]
    
    def compute_single_embedding(self, text: str) -> List[float]:
        embedding = self.model.encode(text, normalize_embeddings=True)
        return embedding.tolist()
    
    def get_vector_size(self) -> int:
        return self.model.get_sentence_embedding_dimension()
    
    def get_model_name(self) -> str:
        return f"SentenceTransformer-{self.model_name.split('/')[-1]}"

# Реализация для BM25
class BM25Model(EmbeddingModel):
    def __init__(self):
        self.corpus = None
        self.bm25 = None
        self.vector_size = 768  # Фиктивное значение для совместимости с Qdrant
        # Подготовка для токенизации и стемминга
        self.stemmer = PorterStemmer()
        try:
            nltk.data.find('tokenizers/punkt')
            nltk.data.find('corpora/stopwords')
        except LookupError:
            nltk.download('punkt')
            nltk.download('stopwords')
        self.stop_words = set(stopwords.words('english'))
    
    def preprocess_text(self, text: str) -> List[str]:
        """Предобработка текста: токенизация, удаление стоп-слов, стемминг."""
        tokens = word_tokenize(text.lower())
        tokens = [self.stemmer.stem(token) for token in tokens if token.isalnum() and token not in self.stop_words]
        return tokens
    
    def initialize_corpus(self, texts: List[str]) -> None:
        """Инициализация индекса BM25 на основе корпуса документов."""
        processed_corpus = [self.preprocess_text(text) for text in texts]
        self.corpus = texts  # Сохраняем оригинальные тексты
        self.bm25 = BM25Okapi(processed_corpus)
    
    def compute_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Для BM25 возвращаем фиктивные эмбеддинги.
        В реальном использовании будем напрямую использовать BM25 для поиска.
        """
        if self.corpus is None:
            self.initialize_corpus(texts)
        
        # Возвращаем фиктивные эмбеддинги для совместимости с API
        return np.random.rand(len(texts), self.vector_size)
    
    def compute_single_embedding(self, text: str) -> np.ndarray:
        """
        Для BM25 эмбеддинг не используется.
        Возвращаем фиктивный эмбеддинг для совместимости с API.
        """
        return np.random.rand(self.vector_size)
    
    def get_vector_size(self) -> int:
        return self.vector_size
    
    def get_model_name(self) -> str:
        return "BM25"
    
    def search(self, query: str, limit: int = 6) -> List[Dict[str, Any]]:
        """Поиск схожих документов с помощью BM25."""
        if self.bm25 is None:
            raise ValueError("BM25 не инициализирован. Сначала вызовите compute_embeddings с корпусом документов.")
        
        query_tokens = self.preprocess_text(query)
        scores = self.bm25.get_scores(query_tokens)
        
        # Получаем индексы документов, отсортированных по убыванию релевантности
        top_indices = np.argsort(scores)[::-1][:limit]
        
        # Формируем результаты в том же формате, что и для других моделей
        results = []
        for i in top_indices:
            if scores[i] > 0:  # Добавляем только документы с ненулевой релевантностью
                results.append({
                    'id': i,
                    'text': self.corpus[i],
                    'score': float(scores[i])
                })
        
        return results

In [27]:
# Модифицированный класс QdrantManager для поддержки разных моделей
class QdrantManager:
    """Класс для работы с базой Qdrant."""
    
    def __init__(self, embedding_model: EmbeddingModel = None):
        self.client = QdrantClient(":memory:")
        self.collection_name = "documentation"
        self.embedding_model = embedding_model
        # Для сохранения оригинальных текстов (нужно для BM25)
        self.corpus = []
        self.id_to_idx = {}  # Отображение id -> индекс в corpus
    
    def set_embedding_model(self, embedding_model: EmbeddingModel) -> None:
        """Устанавливает модель для эмбеддинга."""
        self.embedding_model = embedding_model
    
    def initialize_collection(self, vector_size: int = None):
        """Инициализирует коллекцию в Qdrant."""
        if vector_size is None and self.embedding_model is not None:
            vector_size = self.embedding_model.get_vector_size()
        
        # Удаление существующей коллекции, если она есть
        try:
            self.client.delete_collection(collection_name=self.collection_name)
        except Exception:
            pass
        
        # Создание новой коллекции
        self.client.create_collection(
            collection_name=self.collection_name,
            vectors_config=models.VectorParams(
                size=vector_size,
                distance=models.Distance.COSINE
            )
        )
        
        # Очистка корпуса при инициализации новой коллекции
        self.corpus = []
        self.id_to_idx = {}
    
    def upsert_batch(self, id_offset: int, vectors: List[List[float]], 
                    payloads: List[Dict[str, Any]]) -> None:
        """Добавляет партию данных в Qdrant."""
        # Сохраняем тексты в корпусе (для BM25)
        for idx, payload in enumerate(payloads):
            corpus_idx = len(self.corpus)
            self.corpus.append(payload['text'])
            self.id_to_idx[idx + id_offset] = corpus_idx
        
        # Добавляем в Qdrant
        points = [
            models.PointStruct(
                id=idx + id_offset,
                vector=vector,
                payload=payload
            )
            for idx, (vector, payload) in enumerate(zip(vectors, payloads))
        ]
        
        self.client.upsert(
            collection_name=self.collection_name,
            points=points
        )
    
    def search(self, query: str, limit: int = 6) -> List[Dict[str, Any]]:
        """Выполняет поиск похожих документов."""
        # Для BM25 используем прямой поиск
        if isinstance(self.embedding_model, BM25Model):
            return self.embedding_model.search(query, limit=limit)
        
        # Для других моделей используем Qdrant
        query_vector = self.embedding_model.compute_single_embedding(query)
        search_result = self.client.search(
            collection_name=self.collection_name,
            query_vector=query_vector,
            limit=limit
        )
        
        return [
            {
                'text': hit.payload['text'],
                'id': hit.payload['id'],
                'score': hit.score
            }
            for hit in search_result
        ]

In [28]:
class MetricsCalculator:
    """Класс для расчета метрик оценки качества поиска."""
    
    @staticmethod
    def calculate_recall_at_k(relevant_ids: List[int], retrieved_ids: List[int], k: int) -> float:
        """Рассчитывает Recall@k для одного запроса."""
        if not relevant_ids:
            return 0.0
        
        relevant_retrieved = set(relevant_ids).intersection(set(retrieved_ids[:k]))
        return len(relevant_retrieved) / len(relevant_ids)
    
    @staticmethod
    def calculate_precision_at_k(relevant_ids: List[int], retrieved_ids: List[int], k: int) -> float:
        """Рассчитывает Precision@k для одного запроса."""
        if k == 0 or not retrieved_ids:
            return 0.0
        
        relevant_retrieved = set(relevant_ids).intersection(set(retrieved_ids[:k]))
        return len(relevant_retrieved) / min(k, len(retrieved_ids))
    
    @staticmethod
    def calculate_mrr_at_k(relevant_ids: List[int], retrieved_ids: List[int], k: int) -> float:
        """Рассчитывает MRR@k (Mean Reciprocal Rank) для одного запроса."""
        if not relevant_ids or not retrieved_ids:
            return 0.0
        
        for i, doc_id in enumerate(retrieved_ids[:k]):
            if doc_id in relevant_ids:
                return 1.0 / (i + 1)
        return 0.0
    
    @staticmethod
    def calculate_ndcg_at_k(relevant_ids: List[int], retrieved_ids: List[int], k: int) -> float:
        """Рассчитывает nDCG@k для одного запроса вручную."""
        if not relevant_ids or not retrieved_ids or k <= 0:
            return 0.0
        
        # Обрезаем список до k элементов
        retrieved_ids_k = retrieved_ids[:k]
        
        # Создаем вектор релевантности (1 для релевантных документов, 0 для нерелевантных)
        relevance = [1.0 if doc_id in relevant_ids else 0.0 for doc_id in retrieved_ids_k]
        
        # Если нет релевантных документов среди извлеченных, возвращаем 0
        if sum(relevance) == 0:
            return 0.0
        
        # Вычисляем DCG (Discounted Cumulative Gain)
        dcg = 0.0
        for i, rel in enumerate(relevance):
            # Используем формулу DCG = rel_1 + rel_2/log2(2+1) + rel_3/log2(3+1) + ...
            if rel > 0:
                dcg += rel / np.log2(i + 2)  # +2 потому что индексация с 0, и log2(1)=0
        
        # Вычисляем идеальный DCG (IDCG)
        # В идеальном случае релевантные документы находятся в начале списка
        ideal_relevance = sorted(relevance, reverse=True)
        idcg = 0.0
        for i, rel in enumerate(ideal_relevance):
            if rel > 0:
                idcg += rel / np.log2(i + 2)
        
        # nDCG = DCG / IDCG
        return dcg / idcg if idcg > 0 else 0.0
    
    @staticmethod
    def compute_average_metrics(metrics_list: List[Dict[str, float]]) -> Dict[str, float]:
        """Вычисляет средние значения метрик по всем запросам."""
        if not metrics_list:
            return {}
        
        # Инициализируем результирующий словарь
        result = {}
        
        # Собираем все ключи метрик
        all_keys = set()
        for metrics in metrics_list:
            all_keys.update(metrics.keys())
        
        # Вычисляем среднее значение для каждой метрики
        for key in all_keys:
            values = [metrics.get(key, 0.0) for metrics in metrics_list]
            result[key] = sum(values) / len(metrics_list)
        
        return result

In [29]:
# Модифицированный класс DocumentationQA
class DocumentationQA:
    """Главный класс, объединяющий все компоненты системы."""
    
    def __init__(self, embedding_model: Optional[EmbeddingModel] = None):
        # Если модель не передана, используем SentenceTransformer по умолчанию
        self.embedding_model = embedding_model or SentenceTransformerModel()
        self.qdrant_manager = QdrantManager(self.embedding_model)
        self.metrics_calculator = MetricsCalculator()
        self.is_initialized = False
        self.df = None
        self.section_id_map = {}  # Для хранения мапинга section_content -> id
    
    def set_embedding_model(self, embedding_model: EmbeddingModel) -> None:
        """Устанавливает модель для эмбеддинга и сбрасывает инициализацию."""
        self.embedding_model = embedding_model
        self.qdrant_manager.set_embedding_model(embedding_model)
        self.is_initialized = False
    
    def load_data(self, file_path: str = 'qdrant_documentation_dataset.csv'):
        """Загружает данные из CSV файла."""
        self.df = pd.read_csv(file_path)
        print(f"Загружено {len(self.df)} записей из датасета.")
    
    def initialize_database(self):
        """Инициализирует базу данных, добавляя чанки из датасета в Qdrant."""
        if self.is_initialized:
            return
        
        if self.df is None:
            raise ValueError("Данные не загружены. Вызовите метод load_data() перед инициализацией базы.")
        
        # Получаем уникальные чанки контента
        sections = self.df['section_content'].unique()
        print(f"Найдено {len(sections)} уникальных чанков для индексации.")
        
        # Создаем мапинг section_content -> id
        for idx, section in enumerate(sections):
            self.section_id_map[section] = idx
        
        # Инициализация коллекции Qdrant
        vector_size = self.embedding_model.get_vector_size()
        self.qdrant_manager.initialize_collection(vector_size=vector_size)
        
        # Специальная обработка для BM25
        if isinstance(self.embedding_model, BM25Model):
            self.embedding_model.initialize_corpus(sections)
        
        # Вычисление эмбеддингов и добавление в Qdrant
        batch_size = 100
        for i in range(0, len(sections), batch_size):
            batch = sections[i:i + batch_size]
            
            # Вычисление эмбеддингов
            print(f"Вычисление эмбеддингов для батча {i//batch_size + 1}/{(len(sections) - 1)//batch_size + 1}")
            embeddings = self.embedding_model.compute_embeddings(batch)
            
            # Подготовка payload
            payloads = [
                {
                    'text': section,
                    'id': self.section_id_map[section]
                }
                for section in batch
            ]
            
            # Добавление в Qdrant
            self.qdrant_manager.upsert_batch(i, embeddings, payloads)
        
        self.is_initialized = True
        print(f"База данных успешно инициализирована с моделью {self.embedding_model.get_model_name()}.")
    
    def search_similar_sections(self, query: str, top_k: int = 6) -> List[Dict[str, Any]]:
        """Ищет чанки, похожие на запрос."""
        # Поиск через менеджер Qdrant (он определит нужный метод поиска)
        return self.qdrant_manager.search(query, limit=top_k)
    
    def evaluate_model(self, k_values: List[int] = [1, 4, 6]) -> Dict[str, float]:
        """Оценивает модель по различным метрикам."""
        if not self.is_initialized:
            self.initialize_database()
        
        all_metrics = []
        model_name = self.embedding_model.get_model_name()
        
        print(f"Оценка модели: {model_name}")
        
        # Перебираем все вопросы в датасете
        for idx, row in tqdm(self.df.iterrows(), total=len(self.df), desc="Оценка модели"):
            question = row['question']
            relevant_section = row['section_content']
            relevant_id = self.section_id_map[relevant_section]
            
            # Получаем результаты поиска
            search_results = self.search_similar_sections(question, top_k=max(k_values))
            retrieved_ids = [result['id'] for result in search_results]
            
            # Рассчитываем метрики для текущего запроса
            query_metrics = {}
            for k in k_values:
                query_metrics[f'Recall@{k}'] = self.metrics_calculator.calculate_recall_at_k([relevant_id], retrieved_ids, k)
                query_metrics[f'Precision@{k}'] = self.metrics_calculator.calculate_precision_at_k([relevant_id], retrieved_ids, k)
                query_metrics[f'MRR@{k}'] = self.metrics_calculator.calculate_mrr_at_k([relevant_id], retrieved_ids, k)
                query_metrics[f'nDCG@{k}'] = self.metrics_calculator.calculate_ndcg_at_k([relevant_id], retrieved_ids, k)
            
            all_metrics.append(query_metrics)
        
        # Вычисляем средние метрики
        average_metrics = self.metrics_calculator.compute_average_metrics(all_metrics)
        
        # Добавляем информацию о модели
        average_metrics['model_name'] = model_name
        
        return average_metrics

In [30]:
# Функция для тестирования нескольких моделей и сравнения их результатов
def compare_models(data_path: str = 'qdrant_documentation_dataset.csv', k_values: List[int] = [1, 4, 6]):
    """Сравнивает различные модели и выводит их метрики."""
    # Создаем объект DocumentationQA
    qa_system = DocumentationQA()
    
    # Загружаем данные
    qa_system.load_data(data_path)
    
    # Список моделей для тестирования
    models = [
        SentenceTransformerModel('BAAI/bge-large-en-v1.5'),
        SentenceTransformerModel('intfloat/multilingual-e5-large'),
        BM25Model()
    ]
    
    # Оценка каждой модели
    results = []
    for model in models:
        print(f"\n===== Тестирование модели: {model.get_model_name()} =====")
        
        # Устанавливаем модель
        qa_system.set_embedding_model(model)
        
        # Инициализируем базу данных с новой моделью
        qa_system.initialize_database()
        
        # Оцениваем модель
        metrics = qa_system.evaluate_model(k_values=k_values)
        results.append(metrics)
        
        # Выводим результаты для текущей модели
        print("\nМетрики:")
        for metric_name, value in sorted(metrics.items()):
            if metric_name != 'model_name':
                print(f"{metric_name}: {value:.4f}")
    
    # Сравнительная таблица метрик
    print("\n===== Сравнение моделей =====")
    
    # Получаем все метрики
    all_metric_names = set()
    for result in results:
        all_metric_names.update([k for k in result.keys() if k != 'model_name'])
    
    # Форматируем таблицу
    metric_names_sorted = sorted(all_metric_names)
    model_names = [result['model_name'] for result in results]
    
    # Верхний заголовок таблицы
    header = f"{'Метрика':<15} | " + " | ".join(f"{name:<25}" for name in model_names)
    print(header)
    print("-" * len(header))
    
    # Строки таблицы
    for metric in metric_names_sorted:
        row = f"{metric:<15} | "
        for result in results:
            value = result.get(metric, 0.0)
            row += f"{value:.4f}{' ' * (25 - len(f'{value:.4f}'))}"
            row += " | "
        print(row[:-3])  # Убираем лишний разделитель в конце
    
    return results

In [31]:
def main():
    """Простая функция для вывода всех метрик по каждой модели."""
    # Список моделей для тестирования
    models = [
        SentenceTransformerModel('BAAI/bge-large-en-v1.5'),
        SentenceTransformerModel('intfloat/multilingual-e5-large'),
        BM25Model()
    ]
    
    # Создаем объект DocumentationQA
    qa_system = DocumentationQA()
    
    # Загружаем данные
    qa_system.load_data('qdrant_documentation_dataset.csv')
    
    # Перебираем модели и выводим их метрики
    for model in models:
        print(f"\n===== Модель: {model.get_model_name()} =====")
        
        # Устанавливаем модель
        qa_system.set_embedding_model(model)
        
        # Инициализируем базу данных с новой моделью
        qa_system.initialize_database()
        
        # Оцениваем модель
        metrics = qa_system.evaluate_model(k_values=[1, 4, 6])
        
        # Выводим метрики по группам
        print("\nMetrics:")
        
        # Recall
        print("\nRecall@k:")
        for k in [1, 4, 6]:
            print(f"  Recall@{k}: {metrics.get(f'Recall@{k}', 0.0):.4f}")
        
        # Precision
        print("\nPrecision@k:")
        for k in [1, 4, 6]:
            print(f"  Precision@{k}: {metrics.get(f'Precision@{k}', 0.0):.4f}")
        
        # MRR
        print("\nMRR@k:")
        for k in [1, 4, 6]:
            print(f"  MRR@{k}: {metrics.get(f'MRR@{k}', 0.0):.4f}")
        
        # nDCG
        print("\nnDCG@k:")
        for k in [1, 4, 6]:
            print(f"  nDCG@{k}: {metrics.get(f'nDCG@{k}', 0.0):.4f}")
        
        print("\n" + "="*50)


if __name__ == "__main__":
    main()

Загружено 328 записей из датасета.

===== Модель: SentenceTransformer-bge-large-en-v1.5 =====
Найдено 121 уникальных чанков для индексации.
Вычисление эмбеддингов для батча 1/2
Вычисление эмбеддингов для батча 2/2
База данных успешно инициализирована с моделью SentenceTransformer-bge-large-en-v1.5.
Оценка модели: SentenceTransformer-bge-large-en-v1.5


Оценка модели:   0%|          | 0/328 [00:00<?, ?it/s]C:\Users\sekho\AppData\Local\Temp\ipykernel_25444\1846738923.py:73: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = self.client.search(
Оценка модели: 100%|██████████| 328/328 [00:28<00:00, 11.65it/s]



Metrics:

Recall@k:
  Recall@1: 0.7012
  Recall@4: 0.8811
  Recall@6: 0.9146

Precision@k:
  Precision@1: 0.7012
  Precision@4: 0.2203
  Precision@6: 0.1524

MRR@k:
  MRR@1: 0.7012
  MRR@4: 0.7800
  MRR@6: 0.7865

nDCG@k:
  nDCG@1: 0.7012
  nDCG@4: 0.8058
  nDCG@6: 0.8186


===== Модель: SentenceTransformer-multilingual-e5-large =====
Найдено 121 уникальных чанков для индексации.
Вычисление эмбеддингов для батча 1/2
Вычисление эмбеддингов для батча 2/2
База данных успешно инициализирована с моделью SentenceTransformer-multilingual-e5-large.
Оценка модели: SentenceTransformer-multilingual-e5-large


Оценка модели: 100%|██████████| 328/328 [00:31<00:00, 10.46it/s]



Metrics:

Recall@k:
  Recall@1: 0.6189
  Recall@4: 0.8598
  Recall@6: 0.8933

Precision@k:
  Precision@1: 0.6189
  Precision@4: 0.2149
  Precision@6: 0.1489

MRR@k:
  MRR@1: 0.6189
  MRR@4: 0.7172
  MRR@6: 0.7234

nDCG@k:
  nDCG@1: 0.6189
  nDCG@4: 0.7533
  nDCG@6: 0.7658


===== Модель: BM25 =====
Найдено 121 уникальных чанков для индексации.
Вычисление эмбеддингов для батча 1/2
Вычисление эмбеддингов для батча 2/2
База данных успешно инициализирована с моделью BM25.
Оценка модели: BM25


Оценка модели: 100%|██████████| 328/328 [00:00<00:00, 3205.34it/s]


Metrics:

Recall@k:
  Recall@1: 0.5823
  Recall@4: 0.8262
  Recall@6: 0.8628

Precision@k:
  Precision@1: 0.5823
  Precision@4: 0.2066
  Precision@6: 0.1438

MRR@k:
  MRR@1: 0.5823
  MRR@4: 0.6855
  MRR@6: 0.6923

nDCG@k:
  nDCG@1: 0.5823
  nDCG@4: 0.7213
  nDCG@6: 0.7350

